In [4]:
import urllib.request as request
try:
    wiki_html = request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', timeout=20)
    if wiki_html.status == 200:
        canada_postcode_m = wiki_html.read().decode('utf-8')
    else:
        print(wiki_html.status)
except:
    print('get_error')

# Use *urllib* to load html pages.

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(canada_postcode_m, 'lxml')
postcode_table = soup.table.find_all('td')

# Use *BeatuifulSoup* read all table cells from html
## All postal codes fields are tag with 'td', so just find all td tags.

## Merge 'Borough and Neighbourhood' Fields by Postcode one by one except text is 'Not assigned'

In [6]:
zip_code_dict = {}
for i in range(int(len(postcode_table)/3)):
    postcode = postcode_table[i*3].text
    borough = postcode_table[i*3+1].text
    neighbourhood = postcode_table[i*3+2].text.strip('\n')
    if postcode not in zip_code_dict.keys():
        zip_code_dict[postcode] = {'Borough': '', 'Neighbourhood': []}
    if 'Not assigned' not in borough:
        zip_code_dict[postcode]['Borough'] = borough
    if 'Not assigned' not in neighbourhood:
        zip_code_dict[postcode]['Neighbourhood'].append(neighbourhood)  

# Drop Postcode where Borough is empty
# Fill Neighbourhood with Borough while Neighbourhood is empty.
# Concat Neighbourhood while Neighbourhood counts more than one with comma.

In [11]:
neighbourhood_pd.shape

(103, 3)

In [9]:
neighbourhood_dict = {'Postcode':[], 'Borough': [], 'Neighbourhood': []}
for postcode in zip_code_dict.keys():
    if zip_code_dict[postcode]['Borough'] != '':
        neighbourhood_dict['Postcode'].append(postcode)
        neighbourhood_dict['Borough'].append(zip_code_dict[postcode]['Borough'])
        if len(zip_code_dict[postcode]['Neighbourhood']) > 0:
            neighbourhood_dict['Neighbourhood'].append(','.join(zip_code_dict[postcode]['Neighbourhood']))
        else:
            neighbourhood_dict['Neighbourhood'].append(zip_code_dict[postcode]['Borough'])
import pandas as pd
neighbourhood_pd =  pd.DataFrame(neighbourhood_dict)

# Print the shape.

In [28]:
neighbourhoob_pd_with_GEO.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Because the internet problem from China, I use csv data instead. 
# Use *wget* to retrieve csv data.

In [ ]:
!wget http://cocl.us/Geospatial_data

# Merge to Dataframe and drop extra column.

In [24]:
geo_df = pd.read_csv('Geospatial_data')
neighbourhoob_pd_with_GEO = pd.merge(neighbourhood_pd, geo_df, left_on = ['Postcode'], right_on = ['Postal Code'],how='left')
neighbourhoob_pd_with_GEO.drop(['Postal Code'], axis=1, inplace=True)

# Print the Dataframe Head.